In [1]:
import cv2
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import numpy as np

In [2]:
def get_crop_coordinates(frame, title="Select Crop Area"):
    """
    Function to interactively select a cropping area (region of interest) from an image.

    Parameters:
        frame (numpy.ndarray): The input image frame from which to select the cropping area.
        title (str): The title of the window to be displayed during selection. Default is "Select Crop Area".

    Returns:
        tuple: A tuple (x1, y1, x2, y2) representing the coordinates of the selected cropping area.
               (x1, y1) are the top-left corner, and (x2, y2) are the bottom-right corner of the cropped area.
    """
    r = cv2.selectROI(title, frame, fromCenter=False, showCrosshair=True) #return x,y,w,h
    cv2.destroyAllWindows()
    return r[0], r[1], r[0] + r[2], r[1] + r[3]  

### 1. Import video path

In [18]:
main_folder = r'\\cimec-storage5\acn_lab\shared_acn\Matilde\zebrafish\biodi_experiment\experiment_data\reorganized_video'
subfolder = main_folder + r'\2023-11-10\ID73'
name = r'\2023-11-10-170410_ID73_exp2-n3-speed3-6_TEST'
video_path = Path(subfolder + name + '.avi')

### 2. Open video and check frame

In [19]:
full_video_path = str(video_path) #convert the video path to string
video = cv2.VideoCapture(full_video_path)#read video file and create a video object
framerate = video.get(cv2.CAP_PROP_FPS)
ret, frame = video.read() #read a frame from the video file
    
cv2.imshow("Select First Frame and Crop ROIs", frame) #display the frame
cv2.waitKey(0)
#command to close the window
cv2.destroyAllWindows()

### 3. Select ROIs: number and shape

In [5]:
num_rois = int(input("Enter the number of ROIs to crop: ")) #depending on the number of fish, select the number of ROIs to cut

Enter the number of ROIs to crop: 3


In [6]:
title="Select Crop Area"
roi_coordinates = []

for i in range(num_rois): #Manually crop ROIs
    coordinates = get_crop_coordinates(frame)
    roi_coordinates.append(coordinates)
    cv2.destroyAllWindows()

In [7]:
roi_coordinates #print coordinates

[(35, 116, 440, 877), (434, 110, 808, 872), (803, 109, 1235, 864)]

In [8]:
for i in range(num_rois):
    print(roi_coordinates[i][3] - roi_coordinates[i][1]) #print width of each video 

761
762
755


### 4. Cut the video 

In [20]:
new_folder = video_path.parent.parent.parent / 'cropped_data'
new_folder.mkdir(parents=True, exist_ok=True)

fish_count = 202
output_files = []

for j in range(num_rois):
    fish_count = fish_count + 1
    file_path = new_folder / f"{video_path.stem}_fish{fish_count}.avi" 
    output_files.append(file_path)

fourcc = cv2.VideoWriter_fourcc(*"MJPG") #H264 for mp4, MJPG for avi
outs = [cv2.VideoWriter(str(out_file), fourcc, framerate, (x2 - x1, y2 - y1)) for out_file, (x1, y1, x2, y2) in zip(output_files, roi_coordinates)]

while True:
    ret, frame                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     = video.read()
    if not ret:
        break

    for i, (x1, y1, x2, y2) in enumerate(roi_coordinates):
        cropped_frame = frame[y1:y2, x1:x2]
        outs[i].write(cropped_frame)
        
video.release()
for out in outs:
    out.release()